<a href="https://colab.research.google.com/github/itdusty/blood_cells_classification/blob/main/video_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data preparations

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install lightning
!pip install pytorchvideo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.0/660.0 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.8/764.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute
)
from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [4]:
video_transforms = Compose([
    ApplyTransformToKey(key='video',
    transform=Compose([
        UniformTemporalSubsample(20),
        Normalize((0.45, 0.45, 0.45),(0.225, 0.225, 0.225)),
        # RandomShortSideScale(min_size=248, max_size=256),
        # CenterCropVideo(224),
        RandomHorizontalFlip(p=0.5),
    ]),
    ),
])

In [5]:
import pandas as pd
import numpy as np
import os
import shutil

In [6]:
from torch.utils.data import DataLoader

In [7]:
dataset_path = "/content/drive/MyDrive/Cells classification/prepared_dataset"

In [8]:
os.chdir(dataset_path)
folders = os.listdir()
classes_codes = {}

In [9]:
for i, folder in enumerate(folders):
    classes_codes[folder] = i

In [10]:
classes_codes

{'high_rbc_high_wbc': 0,
 'high_rbc_low_wbc': 1,
 'low_rbc_high_wbc': 2,
 'low_rbc_low_wbc': 3}

In [11]:
dataset_path = "/content/drive/MyDrive/Cells classification/cells_dataset"

In [12]:
def from_path(path):
    import os
    paths = []
    class_names = os.listdir(path)
    for class_name in class_names:
        file_names = os.listdir(f"{path}/{class_name}")
        for file_name in file_names:
            paths.append((f"{path}/{class_name}/{file_name}", {'label': classes_codes[class_name]}))
    return paths

In [ ]:
train_dataset = LabeledVideoDataset(labeled_video_paths=from_path(f"{dataset_path}/train"),
                                    clip_sampler=make_clip_sampler('random', 2),
                                    transform=video_transforms, decode_audio=False)

In [ ]:
train_dataset.num_videos

80

In [17]:
os.chdir("..")
os.getcwd()

'/content/drive/MyDrive/Cells classification'

### Model

In [13]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import torchmetrics

In [34]:
class TestModel(LightningModule):
    def __init__(self, num_classes=2):
        super(TestModel, self).__init__()
        # model architecture
        self.video_model = torch.hub.load("facebookresearch/pytorchvideo", "efficient_x3d_xs", pretrained=True)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(400, num_classes)

        self.lr = 1e-3
        self.batch_size = 64
        self.numworkers = 0
        # evaluation metric
        self.metric = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)
        # loss function
        self.criterion = nn.CrossEntropyLoss()
        # helpers
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.testing_step_outputs = []

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.linear(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr=self.lr)
        scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt, 'lr_scheduler': scheduler}

    def train_dataloader(self):
        dataset = LabeledVideoDataset(labeled_video_paths=from_path(f"{dataset_path}/train"),
                                    clip_sampler=make_clip_sampler('random', 2),
                                    transform=video_transforms, decode_audio=False)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
        return loader

    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        self.training_step_outputs.append({'loss': loss, 'metric': metric})
        return {'loss': loss, 'metric': metric}

    def on_train_epoch_end(self):
        outputs = self.training_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        self.log('train_loss', loss)
        self.log('train_metric', metric)

    def val_dataloader(self):
        dataset = LabeledVideoDataset(labeled_video_paths=from_path(f"{dataset_path}/test"),
                                      clip_sampler=make_clip_sampler('random', 2),
                                      transform=video_transforms, decode_audio=False)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
        return loader

    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        self.validation_step_outputs.append({'loss': loss, 'metric': metric})
        return {'loss': loss, 'metric': metric}

    def on_validation_epoch_end(self):
        outputs = self.validation_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        self.log('val_loss', loss)
        self.log('val_metric', metric)

    def test_dataloader(self):
        dataset = LabeledVideoDataset(labeled_video_paths=from_path(f"{dataset_path}/test"),
                                      clip_sampler=make_clip_sampler('random', 2),
                                      transform=video_transforms, decode_audio=False)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
        return loader

    def test_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        self.testing_step_outputs.append({'label': label, 'pred': out})
        return {'label': label, 'pred': out}

    def on_test_epoch_end(self):
        outputs = self.testing_step_outputs
        label = torch.cat([x['label'] for x in outputs]).cpu().detach().numpy()
        pred = torch.cat([x['pred'].argmax(dim=1) for x in outputs]).cpu().detach().numpy()
        print(classification_report(label, pred))

In [35]:
checkpoint_callback = ModelCheckpoint(monitor="val_loss", dirpath="checkpoints", filename="file", save_last=True)
lr_monitor = LearningRateMonitor(logging_interval="epoch")

In [40]:
model = TestModel(num_classes=4)
seed_everything(0)
trainer = Trainer(max_epochs=20,
                  precision=16,
                  accumulate_grad_batches=2,
                  enable_progress_bar=True,
                  num_sanity_val_steps=0,
                  callbacks=[lr_monitor, checkpoint_callback])

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
INFO:lightning_fabric.utilities.seed:Global seed set to 0
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [41]:
trainer.fit(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type               | Params
---------------------------------------------------
0 | video_model | EfficientX3d       | 3.8 M 
1 | relu        | ReLU               | 0     
2 | linear      | Linear             | 1.6 K 
3 | metric      | MulticlassAccuracy | 0     
4 | criterion   | CrossEntropyLoss   | 0     
---------------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.184    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [42]:
trainer.validate(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.7099999785423279     │
│        val_metric         │    0.7799999713897705     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.7099999785423279, 'val_metric': 0.7799999713897705}]

In [43]:
trainer.test(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

              precision    recall  f1-score   support

           0       0.97      0.68      0.80        47
           1       0.70      0.82      0.75        39
           2       0.70      0.89      0.78        55
           3       0.84      0.73      0.78        59

    accuracy                           0.78       200
   macro avg       0.80      0.78      0.78       200
weighted avg       0.80      0.78      0.78       200



[{}]

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir lightning_logs